In [1]:
import pandas as pd
import numpy as np
import rasterio as rio
import xarray as xr
import rioxarray as xrx
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib.lines import Line2D 
from matplotlib.offsetbox import AnchoredText
import geopandas as gpd
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter

In [2]:
# load sample points
path = 'macander_2017_sample_points'
points = gpd.read_file(f'{path}/macander_sample_points.shp')
points['x'] = points.geometry.x
points['y'] = points.geometry.y
coords = [(x,y) for x, y in zip(points.x, points.y)]